In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install wget
!pip install keras_preprocessing
!pip install keras_tuner

In [5]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Dropout, Bidirectional, Input, Concatenate, concatenate
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
import tensorflow as tf
import keras_tuner as kt
from keras.models import Model

import os.path
import wget
import shutil

In [6]:
def assign_class(label):
    if label == "OR":
        return 0
    else:
        return 1

In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

def evaluate_model(y_prob, y_test):
    y_pred = np.round(y_prob)
    y_true = y_test

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    roc_auc = roc_auc_score(y_test, y_prob, multi_class='ovr')

    metrics = {'accuracy': accuracy,
               'precision': precision,
               'recall': recall,
               'f1': f1,
               'confusion_matrix': cm,
               'roc_auc': roc_auc}

    return metrics

In [ ]:
# import and extract information 

new_train_data = pd.read_csv('/content/drive/MyDrive/swm-data/new_data_final_train.csv')
new_valid_data = pd.read_csv('/content/drive/MyDrive/swm-data/new_data_final_val.csv')
new_test_data = pd.read_csv('/content/drive/MyDrive/swm-data/new_data_final_test.csv')

new_train_data['label_final'] = new_train_data['label'].apply(assign_class)
new_valid_data['label_final'] = new_valid_data['label'].apply(assign_class)
new_test_data['label_final'] = new_test_data['label'].apply(assign_class)

new_train_data.category = pd.factorize(new_train_data.category)[0]
new_test_data.category = pd.factorize(new_test_data.category)[0]
new_valid_data.category = pd.factorize(new_valid_data.category)[0]
category_indices = pd.factorize(new_train_data.category)[1]

x_train_review = new_train_data.text_final
x_train_rating = new_train_data.rating
x_train_category = new_train_data.category
x_train_word_category = new_train_data.word_count_categories

x_test_review = new_test_data.text_final
x_test_rating = new_test_data.rating
x_test_category = new_test_data.category
x_test_word_category = new_test_data.word_count_categories


x_val_review = new_valid_data.text_final
x_val_rating = new_valid_data.rating
x_val_category = new_valid_data.category
x_val_word_category = new_valid_data.word_count_categories


y_train = new_train_data.label_final
y_test = new_test_data.label_final
y_val = new_valid_data.label_final

In [ ]:
# create vocab 

vocab_size = 20000
oov_token = ""
max_length = 200
padding_type = "post"
trunc_type = "post"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(x_train_review) 

x_train_review_pad = pad_sequences(tokenizer.texts_to_sequences(
    x_train_review), maxlen=max_length, padding=padding_type, truncating=trunc_type)
x_test_review_pad = pad_sequences(tokenizer.texts_to_sequences(
    x_test_review), maxlen=max_length, padding=padding_type, truncating=trunc_type)
x_val_review_pad = pad_sequences(tokenizer.texts_to_sequences(
    x_val_review), maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [ ]:
# if not os.path.isfile('/content/drive/MyDrive/swm-data/glove.6B.@00d.txt'):
#     # large file, might take a while to download :)
#     url = 'http://nlp.stanford.edu/data/glove.6B.zip'
#     print('Downloading Pre-trained Word Embeddings')
#     wget.download(url)
#     print('Download Completed!\nUnzipping...')
#     shutil.unpack_archive('glove.6B.zip')

embeddings_index = {}
with open("/content/drive/MyDrive/swm-data/glove.6B.200d.txt") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

num_tokens = len(tokenizer.word_index) + 2
embedding_dim = 200
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Found 400000 word vectors.
Converted 32281 words (8707 misses)


In [ ]:
# create embedding layer 
import keras
from keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [ ]:
def build_model(hp):

    review_branch_input = Input(shape=(200,), name='review_input')
    review_branch = embedding_layer(review_branch_input)
    review_branch = Dropout(0.2)(review_branch)
    review_branch = Bidirectional(
        LSTM(64, dropout=0.2, recurrent_dropout=0))(review_branch)
    review_branch = Dense(64, activation='relu')(review_branch)
    review_branch_output = Dense(16, activation='relu')(review_branch)

    rating_branch_input = Input(shape=(1,), name='rating_input')
    rating_branch = Dense(8, activation='relu')(rating_branch_input)
    rating_branch = Dropout(0.2)(rating_branch)
    rating_branch_output = Dense(16, activation='relu')(rating_branch)

    category_branch_input = Input(
        shape=(1,), name='category_input')
    category_branch = Dense(
        8, activation='relu')(category_branch_input)
    category_branch = Dropout(0.2)(category_branch)
    category_branch_output = Dense(
        16, activation='relu')(category_branch)

    word_category_branch_input = Input(
        shape=(1,), name='word_category_input')
    word_category_branch = Dense(
        8, activation='relu')(word_category_branch_input)
    word_category_branch = Dropout(0.2)(word_category_branch)
    word_category_branch_output = Dense(
        16, activation='relu')(word_category_branch)

    concat = concatenate([review_branch_output, rating_branch_output,
                        category_branch_output, word_category_branch_output], name='Concatenate')

    final_output = Dense(8, activation='relu')(concat)
    final_output = Dense(1, activation='sigmoid')(final_output)

    model = Model(inputs=[review_branch_input, rating_branch_input,
                category_branch_input, word_category_branch_input], outputs=final_output, name='Final_output')

    hp_learning_rate = hp.Float(
        'learning_rate',
        min_value=0.001,
        max_value=0.005,
        sampling='LOG',
        default=1e-3
    )

    # clip value to avoid the gradient exploding
    optimzer = Adam(clipvalue=0.5, learning_rate=hp_learning_rate)

    model.compile(optimizer=optimzer,
                loss='binary_crossentropy',
                metrics=['acc'])

    return model

In [ ]:
tuner = kt.BayesianOptimization(build_model,
                    objective='val_acc',
                    max_trials=5,
                    directory='adl4nlpnew2',
                    project_name='text_classification_bo')

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
tuner.search_space_summary()

Search space summary
Default search space size: 1
learning_rate (Float)
{'default': 0.001, 'conditions': [], 'min_value': 0.001, 'max_value': 0.005, 'step': None, 'sampling': 'log'}


In [ ]:
tuner.search([x_train_review_pad,x_train_rating,x_train_category,x_train_word_category], y_train, epochs=5, 
         validation_data=([x_val_review_pad,x_val_rating,x_val_category,x_val_word_category], y_val),
         callbacks=[stop_early,tf.keras.callbacks.TensorBoard("/tmp/tb_logs")])

Trial 2 Complete [00h 08m 38s]
val_acc: 0.7383131384849548

Best val_acc So Far: 0.7494434714317322
Total elapsed time: 00h 17m 07s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
0.0030188         |0.0012382         |learning_rate

Epoch 1/5
1011/1011 [==============================] - 99s 98ms/step - loss: 0.6285 - acc: 0.6278 - val_loss: 0.5800 - val_acc: 0.6809
Epoch 2/5
1011/1011 [==============================] - 102s 101ms/step - loss: 0.5644 - acc: 0.6990 - val_loss: 0.5285 - val_acc: 0.7245
Epoch 3/5
1011/1011 [==============================] - 96s 95ms/step - loss: 0.5307 - acc: 0.7262 - val_loss: 0.5113 - val_acc: 0.7361
Epoch 4/5
 881/1011 [=========================>....] - ETA: 11s - loss: 0.5081 - acc: 0.7416

In [ ]:
tuner.results_summary()

In [ ]:
best_model = tuner.get_best_models(num_models=1)

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [ ]:
# evaluate and save model 

best_model[0].evaluate([x_test_review_pad, x_test_rating, x_test_category, x_test_word_category], y_test, verbose=2)

In [ ]:
best_model[0].save('/content/drive/MyDrive/swm-data/bilstm_new_tr.h5')

In [ ]:
# Reload the model and verify the accuracy.

model = tf.keras.models.load_model('/content/drive/MyDrive/swm-data/bilstm_new_tr.h5')
model.evaluate([x_test_review_pad, x_test_rating, x_test_category, x_test_word_category], y_test, verbose=2)

127/127 - 2s - loss: 0.5196 - acc: 0.7428 - 2s/epoch - 14ms/step


[0.5196394324302673, 0.742828905582428]

In [ ]:
# Get predictions 
y_prob = model.predict([x_test_review_pad, x_test_rating, x_test_category, x_test_word_category])

# Evaluate the model on the test data
test_metrics = evaluate_model(y_prob, y_test)

# Print the evaluation metrics
print('Test accuracy:', test_metrics['accuracy'])
print('Test precision:', test_metrics['precision'])
print('Test recall:', test_metrics['recall'])
print('Test F1 score:', test_metrics['f1'])
print('Test ROC-AUC score:', test_metrics['roc_auc'])
print('Test confusion matrix:\n', test_metrics['confusion_matrix'])

127/127 [==============================] - 4s 11ms/step
Test accuracy: 0.7428288822947576
Test precision: 0.7045855379188712
Test recall: 0.8119918699186992
Test F1 score: 0.7544853635505194
Test ROC-AUC score: 0.8300968188511364
Test confusion matrix:
 [[1406  670]
 [ 370 1598]]
